# Face Editing

## Prepare Environment

In [ ]:
#@title Download Code { display-mode: "form" } 
import os
os.chdir('/content')
CODE_DIR = 'face-editing'

## clone repo
!git clone https://github.com/estephe-arnaud/face-editing.git $CODE_DIR

## install ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

## change work directory
os.chdir(f'./{CODE_DIR}')

In [ ]:
#@title Download Models { display-mode: "form" } 
!wget "https://nextcloud.univ-lille.fr/index.php/s/8RrH7MzLGFfwMjK/download/pretrained_models.tar.xz" && tar -xvJf ./pretrained_models.tar.xz && rm ./pretrained_models.tar.xz

In [ ]:
#@title Load Face Model { display-mode: "form" } 
from models.face_model import FaceModel
FACE_MODEL = FaceModel()

In [ ]:
#@title Import Modules { display-mode: "form" } 
import os
from PIL import Image
from IPython.display import display

from utils.common import *
from utils.data_utils import *
from utils.inference_utils import *
from utils.editing_utils import *
from utils.morphing_utils import *
from configs.attributes_config import edit_attributes

In [ ]:
#@title Editing Function { display-mode: "form" }
def run(image, edits):
    data = run_prediction(FACE_MODEL, [image], return_weights_deltas=True)
    
    before = data["image_original"][0]
    latent = data["latent"][0]
    weights_deltas = data["weights_deltas"][0]
    style = None
    
    for name, config in edits.items():        
        if config["level"] == "latent":
            direction = config["direction"]
            factor = config["factor"]
            if factor:
                print("Edit {}".format(name))
                latent = edit_latent(latent, direction, factor)
                
        elif config["level"] == "style":
            index = config["index"]
            delta = config["delta"]
            if delta:
                print("Edit {}".format(name))
                if style is None:
                    style = FACE_MODEL.latent2style(latent)
                style = edit_style(style, index, delta)
                
    x = FACE_MODEL.decoder(
        latent=latent, 
        weights_deltas=weights_deltas,
        style=style,
        resize=False
    )
    
    after = tensor2im(x[0])
    return before, after

## Perform Inference

In [ ]:
#@title Load Face Image { display-mode: "form" } 
input_dir = "/content" #@param {type:"string"}

files = image_files(input_dir)

if len(files) > 0:
    files.sort(key=os.path.getmtime)
    f = files[-1]
    
    image = Image.open(f)
    image = exif_transpose(image)
else:
    generator = FACE_MODEL.net.decoder
    image = generate_faces(generator, n_faces=1)[0]

display(image)

In [ ]:
#@title Define Inference Parameters { display-mode: "form" } 
hair = 0 #@param {type:"slider", min:-50, max:50, step:1}
beard_goatee = 0 #@param {type:"slider", min:-20, max:20, step:1}
mouth_smiling = 0 #@param {type:"slider", min:-5, max:5, step:0.5}
mouth_lipstick = 0 #@param {type:"slider", min:-20, max:20, step:1}
eyes_makeup = 0 #@param {type:"slider", min:-25, max:25, step:1}
gaze = 0 #@param {type:"slider", min:-100, max:100, step:5}
eyebrows = 0 #@param {type:"slider", min:-20, max:20, step:1}
gender = 0 #@param {type:"slider", min:-20, max:20, step:1}
pose = 0 #@param {type:"slider", min:-5, max:5, step:0.5}
age = 2.5 #@param {type:"slider", min:-5, max:5, step:0.5}

d = {
    "hair": hair,
    "beard_goatee": beard_goatee,
    "mouth_smiling": mouth_smiling,
    "mouth_lipstick": mouth_lipstick,
    "eyes_makeup": eyes_makeup,
    "gaze": gaze,
    "eyebrows": eyebrows,
    "gender": gender,
    "pose": pose,
    "age": age,    
}

edits = edit_attributes.copy()

for name, config in edits.items():
    if config["level"] == "latent":
        config["factor"] = d[name]
    elif config["level"] == "style":
        config["delta"] = d[name]

before, after = run(image, edits)
display(paste(before, after))